# Addition

In [1]:
import torch
import ttnn

## Open the device

Use `ttnn.open` to get a handle to the device

In [2]:
device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening user mode device driver
2024-01-29 16:40:57.755 | INFO     | SiliconDriver   - Detected 1 PCI device : {0}
2024-01-29 16:40:57.766 | WARNING  | SiliconDriver   - init_detect_tt_device_numanodes(): Could not determine NumaNodeSet for TT device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2024-01-29 16:40:57.766 | WARNING  | SiliconDriver   - Could not find NumaNodeSet for TT Device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2024-01-29 16:40:57.767 | WARNING  | SiliconDriver   - bind_area_memory_nodeset(): Unable to determine TT Device to NumaNode mapping for physical_device_id: 0. Skipping membind.
---- ttSiliconDevice::init_hugepage: bind_area_to_memory_nodeset() failed (physical_device_id: 0 ch: 0). Hugepage allocation is not on NumaNode matching TT Device. Side-Effect is decreased Device->Host perf (Issue #893).
                  Metal | INFO     | AI CLK for device 0 is:

## Configuration

In [3]:
h = 31
w = 22

## Initialize tensors a and b with random values using torch

To create a tensor that can be used by a `ttnn` operation:
1. Create a tensor using torch
2. Use `ttnn.from_torch` to convert the tensor from `torch.Tensor` to `ttnn.Tensor`
3. Copy the tensor onto to the device using `to_device`

In [4]:
torch.manual_seed(0)

torch_a = torch.rand((h, w), dtype=torch.bfloat16)
torch_b = torch.rand((h, w), dtype=torch.bfloat16)

a = ttnn.from_torch(torch_a, layout=ttnn.TILE_LAYOUT, device=device)
b = ttnn.from_torch(torch_b, layout=ttnn.TILE_LAYOUT, device=device)

                     Op | INFO     | Finished Operation ttnn.from_torch                                    in          125109 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           41469 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_layout                                     in          126840 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           29630 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_device                                     in           93529 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           37799 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           24660 nanoseconds
                     Op | INFO    

## Add tensor a and b

`ttnn` supports operator overloading, therefore operator `+` can be used instead of `torch.add`

In [5]:
output = a + b

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           29700 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           23570 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::EltwiseBinary                        in       501037419 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::EltwiseBinary                        in       501149148 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           59050 nanoseconds


## Convert the tensor to ROW_MAJOR_LAYOUT as the output of ttnn.add is in TILE_LAYOUT

In [6]:
output = ttnn.to_layout(output, layout=ttnn.ROW_MAJOR_LAYOUT)

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           31060 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::UntilizeWithUnpadding                in       452708718 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::UntilizeWithUnpadding                in       452808757 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           62359 nanoseconds


## Inspect the result of the add

In [7]:
print(f"shape: {output.shape}")
print(f"dtype: {output.dtype}")
print(f"layout: {output.layout}")
print(f"first row: {output[:1]}")

shape: ttnn.Shape([31, 22])
dtype: DataType.BFLOAT16
layout: Layout.ROW_MAJOR
                     Op | INFO     | Finished Operation ttnn.from_device                                   in           80960 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_torch                                      in          449357 nanoseconds
first row: Tensor([ [0.851562, 0.183594, 0.863281, 1.45312, 0.410156, 1.32812, 1.05469, 1.35156, 0.917969, 1.30469, 0.421875, 1.32031, 0.464844, 0.441406, 1.07812, 0.96875, 0.792969, 1.32031, 0.605469, 1.42969, 1.13281, 0.375]], dtype=bfloat16 )

                     Op | INFO     | Finished Operation torch.Tensor.__getitem__                           in          678126 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_torch                                    in          767105 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_device                                     in           586

## Close the device

Close the handle the device. This is a very important step as the device can hang if not closed properly

In [8]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
